# Assinatura Digital ElGamal — Notebook Didático (com SHA-256)

Este notebook demonstra todas as etapas do esquema de assinatura ElGamal usando SHA-256 como hash:
- Geração de parâmetros
- Geração de chaves
- Assinatura
- Verificação
- Teste com mensagem adulterada

In [ ]:
# Imports e funções auxiliares
import random
from math import gcd
import hashlib

def modinv(a, p):
    t, newt = 0, 1
    r, newr = p, a
    while newr != 0:
        quotient = r // newr
        t, newt = newt, t - quotient * newt
        r, newr = newr, r - quotient * newr
    if r > 1:
        raise ValueError("Inverso não existe")
    if t < 0:
        t = t + p
    return t

In [ ]:
# Parâmetros públicos
p = 467    # Primo (deve ser grande em aplicações reais)
g = 2      # Gerador do grupo
print(f"Parâmetros públicos: p={p}, g={g}")

In [ ]:
# Geração das chaves
x = random.randint(1, p-2)        # Chave privada
y = pow(g, x, p)                  # Chave pública
print(f"Chave privada: x={x}")
print(f"Chave pública: y={y}")

In [ ]:
# Hash usando SHA-256
def hash_mensagem(msg):
    h = hashlib.sha256(msg.encode('utf-8')).hexdigest()
    # Reduz para Z_{p-1} como pede o ElGamal
    return int(h, 16) % (p-1)

In [ ]:
# Função para assinar
def elgamal_sign(msg, p, g, x):
    h = hash_mensagem(msg)
    while True:
        k = random.randint(1, p-2)
        if gcd(k, p-1) == 1:
            break
    r = pow(g, k, p)
    k_inv = modinv(k, p-1)
    s = (k_inv * (h - x * r)) % (p-1)
    return (r, s)

In [ ]:
# Função para verificar
def elgamal_verify(msg, r, s, p, g, y):
    if not (0 < r < p):
        return False
    h = hash_mensagem(msg)
    v1 = (pow(y, r, p) * pow(r, s, p)) % p
    v2 = pow(g, h, p)
    return v1 == v2

In [ ]:
# Exemplo de assinatura
mensagem = "Teste de ElGamal"
assinatura = elgamal_sign(mensagem, p, g, x)
print(f"Assinatura: r={assinatura[0]}, s={assinatura[1]}")

In [ ]:
# Verificação
verif = elgamal_verify(mensagem, assinatura[0], assinatura[1], p, g, y)
print("Assinatura válida?", verif)

In [ ]:
# Teste com mensagem adulterada
mensagem_falsa = "Teste de ElGamal!"
verif2 = elgamal_verify(mensagem_falsa, assinatura[0], assinatura[1], p, g, y)
print("Assinatura válida para mensagem alterada?", verif2)